In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
#allows to print the dataframe nicely
from IPython.core import display as ICD

In [2]:
# import additional packages to insepct data
import pandas as pd
import os 
import random 
from zipfile import ZipFile
import datetime

In [3]:
# import helping functions
#from proj1_helpers import load_csv_data

In [4]:
# import files
data_folder='./data/'
zip_file = ZipFile(data_folder+'all.zip')
# zip file creates a list of files with certain properties
zip_file.infolist()

[<ZipInfo filename='train.csv' compress_type=deflate file_size=48103379 compress_size=17670847>,
 <ZipInfo filename='test.csv' compress_type=deflate file_size=109335584 compress_size=40089137>,
 <ZipInfo filename='sample-submission.csv' compress_type=deflate file_size=5682394 compress_size=1263762>]

Training set, Validation set and Test set.

In [64]:
# now we want to access the 'filename' property in the zipfile variable
# and we create a dictionary of dataframe
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv')}
df_train=dfs['train.csv']
df_test=dfs['test.csv']
df_sample_submission=dfs['sample-submission.csv']

variables prefixed with PRI (for PRImitives) are “raw” quantities about the bunch collision as measured by the detector.
variables prefixed with DER (for DERived) are quantities computed from the primitive features, which were selected by  the physicists of ATLAS
it can happen that for some entries some variables are meaningless or cannot be computed; in this case, their value is −999.0, which is outside the normal range of all variables

In [65]:
df_train_independent=df_train[['Id','DER_mass_MMC','DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h', 'DER_deltar_tau_lep', 
                              'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality', 'PRI_tau_pt', 
                              'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta','PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 
                               'PRI_met_sumet','PRI_jet_num', 'PRI_jet_all_pt' ]]
ICD.display(df_train.columns.values)
df_train_dependent=df_train[['Id','DER_deltaeta_jet_jet','DER_mass_jet_jet', 'DER_prodeta_jet_jet','DER_lep_eta_centrality',
                             'PRI_jet_leading_pt', 'PRI_jet_leading_eta','PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
                             'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi','PRI_jet_num']]
df_train_independent

array(['Id', 'Prediction', 'DER_mass_MMC', 'DER_mass_transverse_met_lep',
       'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet',
       'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_deltar_tau_lep',
       'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau',
       'DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_tau_pt',
       'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta',
       'PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 'PRI_met_sumet',
       'PRI_jet_num', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta',
       'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
       'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi',
       'PRI_jet_all_pt'], dtype=object)

,Id,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,...,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_all_pt
0,100000,138.470,51.655,97.827,27.980,3.064,41.928,197.760,1.582,1.396,...,1.017,0.381,51.626,2.273,-2.414,16.824,-0.277,258.733,2,113.497
1,100001,160.937,68.768,103.235,48.146,3.473,2.078,125.157,0.879,1.414,...,2.039,-3.011,36.918,0.501,0.103,44.704,-1.916,164.546,1,46.226
2,100002,-999.000,162.172,125.953,35.635,3.148,9.336,197.814,3.776,1.414,...,-0.705,-2.093,121.409,-0.953,1.052,54.283,-2.186,260.414,1,44.251
3,100003,143.905,81.417,80.943,0.414,3.310,0.414,75.968,2.354,-1.285,...,-1.655,0.010,53.321,-0.522,-3.100,31.082,0.060,86.062,0,0.000
4,100004,175.864,16.915,134.805,16.405,3.891,16.405,57.983,1.056,-1.385,...,-2.197,-2.231,29.774,0.798,1.569,2.723,-0.871,53.131,0,0.000
5,100005,89.744,13.550,59.149,116.344,1.362,61.619,278.876,0.588,0.479,...,0.371,1.329,31.565,-0.884,1.857,40.735,2.237,282.849,3,193.660
6,100006,148.754,28.862,107.782,106.130,2.941,2.545,305.967,3.371,1.393,...,1.113,2.409,97.240,0.675,-0.966,38.421,-1.443,294.074,2,179.877
7,100007,154.916,10.418,94.714,29.169,2.897,1.526,138.178,0.365,-1.305,...,0.654,1.547,28.740,0.506,-1.347,22.275,-1.761,187.299,1,30.638
8,100008,105.594,50.559,100.989,4.288,2.904,4.288,65.333,0.675,-1.366,...,2.433,-2.532,26.325,0.210,1.884,37.791,0.024,129.804,0,0.000
9,100009,128.053,88.941,69.272,193.392,1.609,28.859,255.123,0.599,0.538,...,-1.533,0.416,32.742,-0.317,-0.636,132.678,0.845,294.741,1,167.735


# Test solving

In [66]:
df_train_independent['PRI_jet_num']=df_train_independent['PRI_jet_num'].astype('float')
prediction=df_train['Prediction']

for column_name in df_train_independent.columns.values:
    
    index_to_drop=df_train_independent.index[np.abs(df_train_independent[column_name]-df_train_independent[column_name].mean()) >= (3*df_train_independent[column_name].std())]
    df_train_independent=df_train_independent.drop(index_to_drop)
    prediction=prediction.drop(index_to_drop)
ICD.display(len(df_train_independent.iloc[:,1:]))
ICD.display(len(prediction))
yb, input_data, ids=np.array(prediction), np.array(df_train_independent.iloc[:,1:]), np.array(df_train_independent['Id'])


/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


212034

212034

In [67]:
ICD.display(input_data)


array([[ 138.47 ,   51.655,   97.827, ...,  258.733,    2.   ,  113.497],
       [ 160.937,   68.768,  103.235, ...,  164.546,    1.   ,   46.226],
       [ 143.905,   81.417,   80.943, ...,   86.062,    0.   ,    0.   ],
       ...,
       [ 105.457,   60.526,   75.839, ...,  198.907,    1.   ,   41.992],
       [  94.951,   19.362,   68.812, ...,  112.718,    0.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ...,   99.405,    0.   ,    0.   ]])

In [68]:
# transform yb into numerical values
yb[np.where(yb=='b')] = -1.
yb[np.where(yb=='s')] = 1.
yb=yb.astype('float')

#Cut dataframe for fast testing
# lines_cut=200000
# tx=input_data[:lines_cut]
# y=yb[:lines_cut]
# y.shape
y=yb
tx=input_data
#input_data.shape

In [69]:
#Import functions
from proj1_helpers import *
from implementations import *

## Stochastic gradient descent 

In [70]:
# Define the parameters of the algorithm.
max_iters = 30
gamma = 0.1
batch_size = 2000

# Initialization
w_initial = np.zeros(tx.shape[1])

# Start SGD.
start_time = datetime.datetime.now()
sgd_losses, sgd_ws = stochastic_gradient_descent(y, tx, w_initial, batch_size, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SGD: execution time={t:.3f} seconds".format(t=exection_time))

SGD(0/29): loss=11423915.156292738, wavg=10.366207799999996, w1=-2.7216643000000005
SGD(1/29): loss=2993696567451630.0, wavg=-181637.11802160117, w1=17966.015733525255
SGD(2/29): loss=9.987530563840285e+23, wavg=3378337726.5052276, w1=-248723073.17398247
SGD(3/29): loss=2.6602177028721255e+32, wavg=-55423755579596.57, w1=3480145075957.5054
SGD(4/29): loss=7.366131816080398e+40, wavg=9.239798825930776e+17, w1=-5.495151837379934e+16
SGD(5/29): loss=2.8022946735841686e+49, wavg=-1.80021600576659e+22, w1=1.1485694907463561e+21
SGD(6/29): loss=7.802365774590365e+57, wavg=3.0078535711791114e+26, w1=-1.7780392737442428e+25
SGD(7/29): loss=2.217676434495682e+66, wavg=-5.073691728388328e+30, w1=2.927365501618012e+29
SGD(8/29): loss=6.883875873725847e+74, wavg=8.934029060930818e+34, w1=-5.318685660607612e+33
SGD(9/29): loss=2.0873989682617072e+83, wavg=-1.5567611349916404e+39, w1=8.967871346683064e+37
SGD(10/29): loss=5.141996853885153e+91, wavg=2.4450239384799256e+43, w1=-1.3551655940823509e+42

## Least Squares

In [71]:
w=least_squares(y,tx)
loss=compute_mse(y, tx, w)
print('Weigths: ',w,"\n\n",'Loss: ',loss)

Weigths:  [ 1.77344302e-04 -9.76874622e-03 -3.74347789e-03 -1.79152344e-03
  1.44794397e-01 -2.77351487e-03 -1.74325514e+00 -5.08625645e-01
  1.01073921e-01  1.74493880e+00 -1.12238830e-03 -1.45555473e-03
  1.76234397e+00  5.52511799e-04  2.22997508e-04  4.15931708e-03
  2.79861046e-05 -5.75654939e-04 -6.05358458e-02  1.74490081e+00] 

 Loss:  0.3392888497340705


In [76]:
tx_test=df_test[['DER_mass_MMC','DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h', 'DER_deltar_tau_lep', 
                              'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality', 'PRI_tau_pt', 
                              'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta','PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 
                               'PRI_met_sumet','PRI_jet_num', 'PRI_jet_all_pt' ]]
y_pred=predict_labels(w,np.array(tx_test))
create_csv_submission(df_test['Id'], y_pred, 'trial1')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568238 entries, 0 to 568237
Data columns (total 32 columns):
Id                             568238 non-null int64
Prediction                     568238 non-null object
DER_mass_MMC                   568238 non-null float64
DER_mass_transverse_met_lep    568238 non-null float64
DER_mass_vis                   568238 non-null float64
DER_pt_h                       568238 non-null float64
DER_deltaeta_jet_jet           568238 non-null float64
DER_mass_jet_jet               568238 non-null float64
DER_prodeta_jet_jet            568238 non-null float64
DER_deltar_tau_lep             568238 non-null float64
DER_pt_tot                     568238 non-null float64
DER_sum_pt                     568238 non-null float64
DER_pt_ratio_lep_tau           568238 non-null float64
DER_met_phi_centrality         568238 non-null float64
DER_lep_eta_centrality         568238 non-null float64
PRI_tau_pt                     568238 non-null float64
PRI_tau_eta 

# Other stuff
Build Polynomial